In [ ]:
from typing import Optional
from datetime import datetime
from sqlmodel import SQLModel, Field

class Production(SQLModel, table=True):
    __tablename__ = "production"   # 明確指定資料表名稱
    __table_args__ = {"schema": "app"}  # 指定 schema

    license_id: str = Field(primary_key=True)
    license_name: Optional[str] = None
    license_info: Optional[str] = None
    exam_date: Optional[str] = None
    price: Optional[str] = None
    exam_location: Optional[str] = None
    registration_start: Optional[datetime] = None
    registration_end: Optional[datetime] = None
    display_status: Optional[int] = None
    created_at: Optional[datetime] = None


In [ ]:
with Session(engine) as session:
    statement = select(Production)
    results = session.exec(statement)
    for hero in results:
        print(hero)

In [ ]:
from app.core import pg_engine
from datetime import datetime

pg = pg_engine.PsqlEngine()#.connect_db()
# pg.connect_db()
# stmt = "UPDATE app.customs SET last_login = NOW() WHERE email = 'hank.yang@omniwaresoft.com.tw';"
table_name = "app.production"
values = [
    (
        '010-789',
        'Linux Professional Institute Linux Essentials',
        '第一筆insert',
        None,
        '120',
        None,
        None,
        None,
        1,
        datetime.now()
    ),
    (
        '010-456',
        'Linux Professional Institute Linux Essentials',
        '第二筆insert',
        None,
        '120',
        None,
        None,
        None,
        1,
        datetime.now()
    )
]
pg.insert_mogrify(table_name, values)
# pg.execute_cmd(stmt)
# pg.close_connect()

In [ ]:
import uuid

user_id = str(uuid.uuid4())
print(user_id)



In [ ]:
import bcrypt

# 設定新密碼
new_password = "123"

# 使用 bcrypt 生成密碼哈希
hashed_password = bcrypt.hashpw(new_password.encode(), bcrypt.gensalt()).decode()

print(hashed_password)
